In [ ]:
# ============ IMPORTS ============
import pandas as pd
import numpy as np
import sys

# Add src folder to path
sys.path.insert(0, 'src')
from src.features import calculate_all_features
from src.scaling import scale_all_features

#  ============ AUTO-RELOAD UPDATED MODULES  ============ 
%load_ext autoreload
%autoreload 2

# ============ LOAD CACHED DATA ============
bitcoin_data_cache = pd.read_pickle('data_cache/bitcoin_data.pkl')
bitcoin_price_history = bitcoin_data_cache['bitcoin_price_history']
fear_greed_index_data = bitcoin_data_cache['fgi_data']
start_date = bitcoin_data_cache['start_date']
end_date = bitcoin_data_cache['end_date']

print(f"Loaded from {bitcoin_data_cache['cache_date']}")

# ============ CALCULATE ALL FEATURES ============
# This contains BOTH raw OHLCV AND engineered features
bitcoin_price_and_features = calculate_all_features(bitcoin_price_history, fear_greed_index_data)

# ============ PREPARE DATA FOR DASHBOARD ============
# Separate raw OHLCV columns from engineered features
ohlcv_columns = [
    col for col in bitcoin_price_and_features.columns
    if isinstance(col, tuple) and len(col) == 2 and col[1] == 'BTC-USD'
]

# Extract only engineered features (no raw OHLCV) for scaling
engineered_features = bitcoin_price_and_features.drop(ohlcv_columns, axis=1, errors='ignore')

# Flatten MultiIndex if present
if isinstance(engineered_features.columns, pd.MultiIndex):
    engineered_features.columns = engineered_features.columns.get_level_values(0)

# Scale the engineered features
scaled_features = scale_all_features(engineered_features)

# ============ OUTPUT DATAFRAMES ============
features_abs = scaled_features['features_abs']  # Scaled features (absolute)
features_norm = scaled_features['features_norm']  # Scaled features (0-1)
pd.to_pickle({
    'bitcoin_price_and_features': bitcoin_price_and_features,
    'features_abs': features_abs,
    'features_norm': features_norm
}, 'data_cache/processed_data.pkl')
